In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
from core.reconstruction import MLEvaluator, MLReconstructorBase
import core
import keras


PLOTS_DIR = "plots/plot_histories/"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)


load_config = get_load_config_from_yaml("workspace_config.yaml")
load_config_high_level = get_load_config_from_yaml(
    "workspace_config_high_level_features.yaml"
)
DataProcessor = DataPreprocessor(load_config)
DataProcessor_high_level = DataPreprocessor(load_config_high_level)

data_config_path = "workspace_config.yaml"
with open(data_config_path, "r") as file:
    data_configs = yaml.safe_load(file)

data_config = DataProcessor.load_data(
    data_configs["data_path"]["nominal"], "reco", max_events=400000
)
X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)
data_config_high_level = DataProcessor_high_level.load_data(
    data_configs["data_path"]["nominal"], "reco", max_events=400000
)
X_train_high_level, y_train_high_level, X_val_high_level, y_val_high_level = DataProcessor_high_level.split_data(
    test_size=0.1
)
MODEL_DIRS = ["models/assignment_rnn/", "models/assignment_transformer/", "models/assignment_transformer_high_level_features/"]
MODEL_CONFIGS = [data_config, data_config, data_config_high_level]
MODEL_NAMES = ["RNN", "Transformer", "Transformer High Level"]
MODEL_X_TEST = [X_val, X_val, X_val_high_level]
MODEL_Y_TEST = [y_val, y_val, y_val_high_level]

ml_reconstructors = []
ml_evaluators = []
for i in range(len(MODEL_DIRS)):
    reconstructor = MLReconstructorBase(MODEL_CONFIGS[i], name=MODEL_NAMES[i], perform_regression=False)
    reconstructor.load_model(MODEL_DIRS[i] + "model.keras")
    ml_reconstructors.append(reconstructor)

evaluator = MLEvaluator(
    reconstructor=ml_reconstructors,
    X_test=MODEL_X_TEST,
    y_test=MODEL_Y_TEST,
)



In [ ]:
fig, ax = evaluator.plot_training_history()